In [1]:
import yfinance as yf
from requests import Session
from requests_cache import CacheMixin, SQLiteCache
from requests_ratelimiter import LimiterMixin, MemoryQueueBucket
from pyrate_limiter import Duration, RequestRate, Limiter

# Define a session with caching and rate limiting
class CachedLimiterSession(CacheMixin, LimiterMixin, Session):
    pass

session = CachedLimiterSession(
    limiter=Limiter(RequestRate(3, Duration.SECOND*5)),  # max 2 requests per 5 seconds
    bucket_class=MemoryQueueBucket,
    backend=SQLiteCache("yfinance.cache"),
)

# # Add a custom User-agent header
# session.headers['User-agent'] = 'my-program/1.0'

def getEPS(tickers):
    earnings_dict = {}
    for ticker in tickers:
        try:
            comp = yf.Ticker(ticker)
            eps_data = comp.get_earnings_dates(limit=25)
            
            if eps_data is None or eps_data.empty:
                raise ValueError(f"No data found for {ticker}")
            
            company_dict = {}
            for date, row in eps_data.iterrows():
                if all(key in row for key in ['EPS Estimate', 'Reported EPS', 'Surprise(%)']):
                    if int(date.strftime('%Y%m%d'))<20240601:
                        company_dict[date.strftime('%Y%m%d')] = {
                            "EPS Estimate": float(row['EPS Estimate']),
                            "Reported EPS": float(row['Reported EPS']),
                            "Surprise(%)": float(row['Surprise(%)'])
                        }
                else:
                    raise ValueError(f"Missing data for {ticker} on {date}")
            earnings_dict[ticker] = company_dict
        except Exception as e:
            print(f"Error for ticker {ticker}: {e}")

    return earnings_dict

In [2]:
import pymupdf
import os

def getTicker(inFilePath):
    doc = pymupdf.open(inFilePath) # open a document
    text = doc[0].get_text('blocks', sort=True)[1][4]
    s,e= 0,0
    started = False
    for i in range(len(text)):
        if text[i] == '(':
            s = i+1
            started = True
        if started and text[i]==' ':
            e = i
            break
    return text[s:e]

In [3]:
import pandas as pd
from datetime import datetime, timedelta
import numpy as np

# Replace 'file.csv' with the path to your CSV file
df = pd.read_excel('UtilitiesNames.xlsx')

fileNames = df['fileName'] 
inFolderPath = folder_path = os.getcwd() + '/Pdfs'

TickerToFiles = {}

for fileName in fileNames[:]:
    pdfFileName = fileName[:-11]+'.pdf' 
    inFilePath = os.path.join(inFolderPath,pdfFileName)
    ticker = getTicker(inFilePath)
    if ticker in TickerToFiles:
        TickerToFiles[ticker].append(fileName)
    else: 
        TickerToFiles[ticker] = [fileName]


tickers = list(TickerToFiles.keys())
print(len(tickers))
# # # print(getEPS(tickers[:180]), '\n')

yf_eps = getEPS(tickers)









54


CWEN/A: $CWEN/A: possibly delisted; No earnings dates found


Error for ticker CWEN/A: No data found for CWEN/A


In [ ]:
nan = ''

In [6]:
tickers = ['CWEN-A']
yf_eps = getEPS(tickers)

In [ ]:
yf_eps  =

In [7]:
eps_data = {
    'fileName': [],
    'date': [],
    'ticker': [],
    "EPS Estimate": [],
    "Reported EPS": [],
    "Surprise(%)": [],
}

noEPS = {}
for ticker, files in TickerToFiles.items():
    if ticker not in yf_eps:
        noEPS[ticker] = files
        continue
    for fileName in files:
        date = fileName[:8]
        date_obj = datetime.strptime(date, '%Y%m%d')
        dayBefore = (date_obj - timedelta(days=1)).strftime('%Y%m%d')
        dayAfter = (date_obj + timedelta(days=1)).strftime('%Y%m%d')
        if date in yf_eps[ticker]:
            eps_data['fileName'].append(fileName)
            eps_data['date'].append(date)
            eps_data['ticker'].append(ticker)
            eps_data["EPS Estimate"].append(yf_eps[ticker][date]["EPS Estimate"])
            eps_data["Reported EPS"].append(yf_eps[ticker][date]["Reported EPS"])
            eps_data["Surprise(%)"].append(yf_eps[ticker][date]["Surprise(%)"])
            continue
        elif dayBefore in yf_eps[ticker]:
            eps_data['fileName'].append(fileName)
            eps_data['date'].append(date)
            eps_data['ticker'].append(ticker)
            eps_data["EPS Estimate"].append(yf_eps[ticker][dayBefore]["EPS Estimate"])
            eps_data["Reported EPS"].append(yf_eps[ticker][dayBefore]["Reported EPS"])
            eps_data["Surprise(%)"].append(yf_eps[ticker][dayBefore]["Surprise(%)"])
            continue
        elif dayAfter in yf_eps[ticker]:
            eps_data['fileName'].append(fileName)
            eps_data['date'].append(date)
            eps_data['ticker'].append(ticker)
            eps_data["EPS Estimate"].append(yf_eps[ticker][dayAfter]["EPS Estimate"])
            eps_data["Reported EPS"].append(yf_eps[ticker][dayAfter]["Reported EPS"])
            eps_data["Surprise(%)"].append(yf_eps[ticker][dayAfter]["Surprise(%)"])
            continue
        else:
            if ticker in noEPS:
                noEPS[ticker].append(fileName)
            else:
                noEPS[ticker] = [fileName]

print(noEPS)

{'AVA': ['20230802_Avista_Corp-_Earnings_Call_2023-8-02_DN000000003018188924_RESULT.txt', '20231101_Avista_Corp-_Earnings_Call_2023-11-01_RT000000003024937986_RESULT.txt', '20230503_Avista_Corp-_Earnings_Call_2023-5-03_DN000000003010484448_RESULT.txt', '20240221_Avista_Corp-_Earnings_Call_2024-2-21_DN000000003032183886_RESULT.txt'], 'SRE': ['20240227_Sempra-_Earnings_Call_2024-2-27_RT000000003032716095_RESULT.txt', '20230803_Sempra-_Earnings_Call_2023-8-03_RT000000003018321077_RESULT.txt', '20231103_Sempra-_Earnings_Call_2023-11-03_RT000000003025328657_RESULT.txt', '20230504_Sempra-_Earnings_Call_2023-5-04_DN000000003010629366_RESULT.txt'], 'LNT': ['20240216_Alliant_Energy_Corp-_Earnings_Call_2024-2-16_DN000000003034401198_RESULT.txt', '20230505_Alliant_Energy_Corp-_Earnings_Call_2023-5-05_DN000000003010728542_RESULT.txt', '20230804_Alliant_Energy_Corp-_Earnings_Call_2023-8-04_RT000000003018579648_RESULT.txt', '20231103_Alliant_Energy_Corp-_Earnings_Call_2023-11-03_DN000000003025260038

In [8]:
df = pd.read_excel('UtilitiesNamesEPS.xlsx')

eps_df = pd.DataFrame(eps_data)

# Ensure the date columns are strings
df['date'] = df['date'].astype(str)
eps_df['date'] = eps_df['date'].astype(str)

# Ensure the fileName columns are strings
df['fileName'] = df['fileName'].astype(str)
eps_df['fileName'] = eps_df['fileName'].astype(str)

# Merge the original DataFrame with the EPS DataFrame on 'fileName' and 'date'
merged_df = pd.merge(df, eps_df, how='left', left_on=['fileName', 'date'], right_on=['fileName', 'date'])

# Save the merged DataFrame to a new Excel file
output_file_path = 'UtilitiesNamesEPS.xlsx'
merged_df.to_excel(output_file_path, index=False)

output_file_path

'UtilitiesNamesEPS.xlsx'

Tests Below

In [ ]:
print(getEPS(['DAN']))

In [9]:
import yfinance as yf
comp = yf.Ticker('CWEN-A')
eps_company_data = comp.get_earnings_dates(limit=23)

print(eps_company_data)


                           EPS Estimate  Reported EPS  Surprise(%)
Earnings Date                                                     
2025-05-07 08:00:00-04:00           NaN           NaN          NaN
2025-02-20 08:00:00-05:00           NaN           NaN          NaN
2024-10-31 08:00:00-04:00           NaN           NaN          NaN
2024-08-01 08:00:00-04:00          0.76           NaN          NaN
2024-05-09 06:00:00-04:00         -0.25         -0.02       0.9192
2024-02-22 06:00:00-05:00          0.04          0.31       7.2667
2023-11-02 06:00:00-04:00          0.66          0.03      -0.9542
2023-08-08 06:00:00-04:00          0.60          0.33      -0.4515
2023-05-04 06:00:00-04:00         -0.37           NaN       1.0000
2023-02-23 06:00:00-05:00         -0.88          0.10       1.1141
2022-11-02 06:00:00-04:00          0.34          0.28      -0.1813
2022-08-02 06:00:00-04:00          0.46          4.89       9.5844
2022-05-05 06:00:00-04:00         -0.07         -0.28      -3.